In [1]:
import tensorflow as tf

2022-11-15 17:44:00.892483: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
seq_embed=tf.random.normal([5,10,3])

2022-11-15 17:44:03.783319: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-11-15 17:44:03.787769: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-15 17:44:03.787798: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-11-15 17:44:03.788338: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
att=tf.random.normal([5,2,10])

In [4]:
def contrastive_label(seq_embed,s_item_index,k):
    #s_item_emd,s_item_index=tf.math.top_k(att, k)
    s_item_index=tf.reshape(s_item_index,[-1,])
    h_index=tf.sort(tf.tile(tf.reshape(tf.range(tf.shape(seq_embed)[0]),[-1,]),[interest_num*k]),axis=0)
    index = tf.stack([h_index, s_item_index])
    #print(index)
    index=tf.transpose(index,[1,0])
    index=tf.reshape(index,[-1,k,2])
    #print(index)
    all_user_label=tf.gather_nd(seq_embed, index)
    all_user_label=tf.reshape(all_user_label,[-1,interest_num,k,hidden_size])
    all_user_label=tf.reduce_mean(all_user_label,axis=-2)
    return all_user_label

In [11]:
k=3
interest_num=2
hidden_size=3
att_sum_dict={}
att_sum_mask={}
all_user_label_dict={}
for i in range(2,k+1):
    s_item_emd,s_item_index=tf.math.top_k(att, i)
    att_sum_dict['att_sum_'+str(i)]=tf.reduce_sum(s_item_emd,axis=-1)
    att_sum_mask['att_sum_mask_'+str(i)]=tf.math.greater(att_sum_dict['att_sum_'+str(i)],0.5)
    #masked_seq_emb=tf.boolean_mask(seq_embed,att_sum_mask['att_sum_mask_'+str(i)])
    #masked_s_item_index=tf.boolean_mask(s_item_index,att_sum_mask['att_sum_mask_'+str(i)])
    all_user_label_dict['all_user_label_'+str(i)]=contrastive_label(seq_embed,s_item_index,i)
all_user_label=tf.zeros_like(all_user_label_dict['all_user_label_'+str(2)])
ones=tf.ones_like(att_sum_mask['att_sum_mask_'+str(2)],dtype=tf.float32)
att_mask=tf.zeros_like(ones,dtype=tf.float32)
for i in range(2,k+1):
    all_user_label+=all_user_label_dict['all_user_label_'+str(i)]
    if i==k:
        att_mask+=ones
    else:
        att_mask+=tf.cast(att_sum_mask['att_sum_mask_'+str(i)],tf.float32)
    #print(all_user_label_dict['masked_'+'all_user_label_'+str(i)])

all_user_label=tf.multiply(all_user_label,tf.expand_dims(1/att_mask,axis=-1))

In [12]:
print(all_user_label)

tf.Tensor(
[[[ 0.39935762 -0.81428874  0.47094595]
  [ 0.44576967  0.26233178  0.37415892]]

 [[-0.21408974 -0.09300266 -0.38400415]
  [-0.33920738 -0.8794616   0.18464765]]

 [[ 0.7482717   0.66177714  0.8963134 ]
  [-0.78079736 -1.08051     0.84546244]]

 [[-0.91181743 -0.2474732  -0.12092203]
  [-0.0108271   0.87687534 -0.44777787]]

 [[ 0.6875379   0.8822533   1.1960871 ]
  [-0.00301336 -0.04612119 -0.7223369 ]]], shape=(5, 2, 3), dtype=float32)


In [13]:
1/att_mask

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 1. ],
       [0.5, 0.5],
       [0.5, 0.5]], dtype=float32)>

In [5]:
import tensorflow as tf
x = tf.constant([[2, 4],[3,0]])
y = tf.constant([2, 4])
tf.math.equal(x, 2)

<tf.Tensor: shape=(2, 2), dtype=bool, numpy=
array([[ True, False],
       [False, False]])>

In [8]:
tf.cast(tf.cast(x,dtype=tf.bool),dtype=tf.float32)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 0.]], dtype=float32)>